# Update Products

Sometimes you need to update Products

**DO NOT RUN THIS WHOLE SCRIPT** It does all sorts of random things, you don't want to do all of them. It is meant to be run a cell at a time depending on what you want.

In [ ]:
import logging
import requests
import json
import pandas as pd

# Start logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logging.debug('Start of program')

from pyLightspeed.lsretail import api

## Get Keys and A Connection
This example uses a file you will have to create on your local machine to store keys. This is not best practice - you should probably store your keys in environment variables - but is simpler for people just trying to test and play. See the example files in example\data and adjust with your information.

In [ ]:
KEY_FILE = "D:\Development\.keys\lightspeed_keys.json"

with open(KEY_FILE) as f:
    keys = json.load(f)

store_data = {
            'account_id': keys["account_id"],
            'save_path': 'D:\\Development\\.keys\\'
            }

credentials = {
            'client_id': keys["client_id"],
            'client_secret': keys["client_secret"]
            }

# Creates the connection to lightspeed, and returns a connection object with useful properties
lsr = api.Connection(store_data, credentials)

## Copy System Sku to Custom Sku

What fun! There is no accessible common key between LS Retail and eCom, so if you want to do anything with the data outside of LS, you need to create your own. So copy the system sku to Custom Sku, which will then be carried to eCom in the Article Code.

In [ ]:
    items = lsr.list('Item', filter = 'customSku=~,')

    for item in items:
        data = {"customSku": item["systemSku"]}
        lsr.update('Item', item['itemID'], data)
        logging.debug(f"Updated {item['itemID']} {item['description']}")


## Delete all EANs
Someone was using the EAN field for garbage, but the gargbage gets picked up by your Google shopping feed and causes failures. So have to delete all the EANs (assuming you are not using them for real stuff)

In [ ]:
    items = lsr.list("Item", filter = 'ean=!~,')
    
    for item in items:
        data = {"ean": None}
        lsr.update("Item", item["itemID"], data)
        logging.debug(f'Updated {item["itemID"]}')

## Experiments

Below here, just me messing.

In [ ]:
#filter = load_relations=["Contact"]&Contact.email=%2C


url = f'https://api.lightspeedapp.com/API/Account/{lsr.account_id}/Item.json?&customSku=!~,%2%'


response = requests.request('GET', url, headers=lsr.headers)

response.json()

In [ ]:
response.json()['@attributes']['count']


In [ ]:
url = f'https://api.lightspeedapp.com/API/Account/{lsr.account_id}/en/resources.json'
response = requests.request('GET', url, headers=lsr.headers)

In [ ]:
response.json()
